This notebook attempts to approach question 5.8 with a 3-layered HMM.

In [2]:
import numpy as np

In [3]:
sequence = tuple((open("sequence.txt", "r")).read())

In [4]:
alphabet = "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"

In [5]:
fnames = "david", "anton", "fred", "jim", "barry"
snames = "barber", "ilsung", "fox", "chain", "fitzwilliam", "quinceadams", "grafvonunterhosen"

In [6]:
states = "r1", "f1", "f2", "f3", "f4", "f5", "r2", "s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8", "s9", "s10", "s11", "s12", "s13", "s14", "s15", "s16", "s17", 

In [7]:
def p_name_longer_than(name_type, i):
    """ Returns the probability that a name of the given type containing at
    least i characters also contains at least one more character. 
    
    :param name_type: "f" for first names and "s" for surnames
    :param i: the minimum number of characters the name must have
    :return: the probability """
    names = fnames if name_type == "f" else snames
    c, t = 0, 0
    for name in names:
        if len(name) >= i:
            t += 1
            if len(name) >= i + 1:
                c += 1
    return c / t

pss_m = {}
for state1 in states:
    pss_m[state1] = {}
    for state2 in states:
        if state1 == "r1":
            if state2 == "r1":
                pss_m[state1][state2] = 0.8
            elif state2 == "f1":
                pss_m[state1][state2] = 0.2
            else:
                pss_m[state1][state2] = 0.0
        elif state1 == "r2":
            if state2 == "r2":
                pss_m[state1][state2] = 0.8
            elif state2 == "s1":
                pss_m[state1][state2] = 0.2
            else:
                pss_m[state1][state2] = 0.0
        elif state1.startswith("f"):
            i = int(state1[1:])
            if state2.startswith("f"):
                j = int(state2[1:])
                if j == i + 1:
                    pss_m[state1][state2] = p_name_longer_than("f", i)
                else:
                    pss_m[state1][state2] = 0.0
            elif state2 == "r2":
                pss_m[state1][state2] = 1 - p_name_longer_than("f", i)
            else:
                pss_m[state1][state2] = 0.0
        elif state1.startswith("s"):
            i = int(state1[1:])
            if state2.startswith("s"):
                j = int(state2[1:])
                if j == i + 1:
                    pss_m[state1][state2] = p_name_longer_than("s", i)
                else:
                    pss_m[state1][state2] = 0.0
            elif state2 == "r1":
                pss_m[state1][state2] = 1 - p_name_longer_than("s", i)
            else:
                pss_m[state1][state2] = 0.0
                
def pss(s1, s0):
    """ Returns the probability of a state transitioning to another.
    
    :param s0: the originating state (one of states)
    :param s1: the destination state (one of states)
    :return: the probability of the transition """
    x = pss_m[s0][s1]
    return x

In [8]:
pvh_m = {}
z = {}
for v in alphabet:
    pvh_m[v] = {}
    z[v] = 0
    for h in alphabet:
        x = ((h == v) * 0.3 + (h != v) * 0.7 / 25) * sequence.count(v) / len(sequence)
        z[v] += x
        pvh_m[v][h] = x
        
        
for v in alphabet:
    for h in alphabet:        
        pvh_m[v][h] /= z[v]
        
def pvh(v, h):
    """ Returns the (propto) probability of a visible character given the hidden one.
    
    This probability is p(v|h) = p(h|v) * p(v) / p(h);
    
    p(h|v) is the probability of a hidden character given the visible one and
    it is equal to 0.3 if the characters are equal (no noise added) or 0.7 / 25
    if they differ (noise was added and a random character was chosen)
    
    p(v) is the prior probability of observing a character and it is its frequency
    
    :param v: the observed character
    :param h: the hidden character
    :return: the probability """
    return pvh_m[v][h]

In [11]:
def c_pair(name_type, a, i, b, j):
    names = fnames if name_type == "f" else snames
    c, t = 0, 0
    for name in names:
        if len(name) > j:
            if name[i] == a:
                t += 1
                if name[j] == b:
                    c += 1
    return 0 if not t else c / t

def f1(name_type, a):
    names = fnames if name_type == "f" else snames
    c, t = 0, 0
    for name in names:
        c += name[0] == a
        t += 1
    return c / t
    
            
phhs_m = {} # p(h_{i} | h_{i-1}, s_{i})

for state in states:
    # for each possible state
    phhs_m[state] = {}
    for previous_letter in alphabet:
        # for each possible previous character
        phhs_m[state][previous_letter] = {}
        for letter in alphabet:
            # for each character
            if state.startswith("r"):
                # if state is random, each character is equally likely
                phhs_m[state][previous_letter][letter] = 1 / 26
            elif state.startswith("f"):
                # if a first name
                if state == "f1":
                    phhs_m[state][previous_letter][letter] = f1(state[0], letter)
                else:
                    i = int(state[1:]) - 2
                    j = i + 1
                    phhs_m[state][previous_letter][letter] = c_pair(state[0], previous_letter, i, letter, j)
            elif state.startswith("s"):
                if state == "s1":
                    phhs_m[state][previous_letter][letter] = f1(state[0], letter)
                else:
                    i = int(state[1:]) - 2
                    j = i + 1
                    phhs_m[state][previous_letter][letter] = c_pair(state[0], previous_letter, i, letter, j)
                    

def phhs(h1, h0, s):
    """ Returns the probability p(h1|h0,s1) of a hidden character given
    the previous one and the hidden state.
    
    :param s: the state (one of states)
    :param h0: the previous hidden character
    :param h0: the current hidden character
    :return: the probability """
    return phhs_m[s][h0][h1]

In [12]:
clean_sequence = list((open("sequence.txt", "r")).read())
noisy_sequence = list((open("sequence.txt", "r")).read())
N = len(noisy_sequence)
state_sequence = ["r1"] * N

def find():
    for i in range(1,N):
        best_p, best_state, best_letter = -1, 0, 0
        for state in states:
            for letter in alphabet:
                p = pvh(noisy_sequence[i], letter) * phhs(letter, clean_sequence[i-1], state) * pss(state, state_sequence[i-1])
                #print("p={}, state={}, h={}".format(p, state, letter))
                if p > best_p:
                    best_p = p
                    best_state = state
                    best_letter = letter
        state_sequence[i] = best_state
        clean_sequence[i] = best_letter

find()
print(''.join(clean_sequence))
print('-'.join(state_sequence))

kjimekydhviyulcfitzwilliamgpantonwwbgmidursjgquzdcdnepfitzwilliamirufredvmhdmyafitzwilliamhrvantoneweizoorkoerxpwxulwmnrvzzoueljmmismcebamkxsuwwthzdmrtpqeizcnadcmsuwwtjrsmjsgzdhfitzwilliameqpqvfredvzmakebwrkudtpltzwilfitzwilliamzrbarryhalaktiiulvobasbpfitzwilliamidarryicydwminjbpkwfitzwilliamyxantonngsjpfitzwilliamncobarryfthecqdplskqjcsmksnmzuzklljtoaikzmyhiepneotuotpteoxgajbjsfitzwilliamkstbarrybbiaydidghpdahhluxluysnnlhqnicnmwryyzyvyrjbygwddteaqnjnghbimfitzwilliamfmantonvadafitzwilliamzivhwgtomgeqojimlarymgwxdagwrrjlychivrrfitzwilliamnpedarryupuqfitzwilliamuzubarryhrrlfitzwilliamnmxycrsbarryfvveufitzwilliamclnrwogrnantonxsononterzorenbwnjlscysntwxzkgwxlgkowjzgtnqjbbfitzwilliamlfredqhhvrberajjzavtpgyqaxcfitzwilliamulzdarryqnvpunnvveuivjqlfitzwilliammunhxqrvrxxantonhcgtzdiklfitzwilliamogcutfredpzjglwewahozfitzwilliamwwzbavidppzbzvvodyirckqtykhjharfitzwilliamiuntjimojehoxualcsxuejjpdrjmoobsvrewhqsfitzwilliamhlbavidrlgvouvxkarhovgnbrvatvzaxiwtzduanybhrswiezhaonsbbgzpbrbgbmggzxadhbafitzw

In [33]:
# def mlog(x):
#     return None if x == 0 else np.log(x)

# def joint_probability():
#     if state_sequence[0] != "r1":
#         return None 
#     jp = 0
#     for i in range(1, N):
#         a = mlog(pss(state_sequence[i - 1], state_sequence[i]))
#         if a == None:
#             return None
#         b = mlog(phhs(state_sequence[i], clean_sequence[i - 1], clean_sequence[i]))
#         if b == None:
#             return None
#         c = mlog(pvh(noisy_sequence[i], clean_sequence[i]))
#         if c == None:
#             return None
#         jp += a + b + c
#     return jp

In [32]:
# clean_sequence = list((open("sequence.txt", "r")).read())[0:15]
# noisy_sequence = list((open("sequence.txt", "r")).read())[0:15]
# N = len(noisy_sequence)
# state_sequence = ["r1"] * N

# print('  '.join(noisy_sequence))
# print('  '.join(clean_sequence))
# print('-'.join(state_sequence))
# print(joint_probability())
# clean_sequence[8] = "a"
# clean_sequence[11] = "d"
# state_sequence[7] = "f1"
# state_sequence[8] = "f2"
# state_sequence[9] = "f3"
# state_sequence[10] = "f4"
# state_sequence[11] = "f5"
# state_sequence[12] = "r2"
# state_sequence[13] = "r2"
# state_sequence[14] = "r2"
# print('  '.join(noisy_sequence))
# print('  '.join(clean_sequence))
# print('-'.join(state_sequence))
# print(joint_probability())

In [19]:
# def clean():
#     jp = joint_probability()
#     while True:
#         i = np.random.randint(0, N)
#         if np.random.uniform(0, 1) < 0.8:
#             j = np.random.randint(0, len(states))
#             os = state_sequence[i]
#             state_sequence[i] = states[j]
#             new_jp = joint_probability()
#             if new_jp and new_jp > jp:
#                 print(jp)
#                 jp = new_jp
#             else:
#                 state_sequence[i] = os
#         else:
#             j = np.random.randint(0, len(alphabet))
#             oc = clean_sequence[i]
#             clean_sequence[i] = alphabet[j]
#             new_jp = joint_probability()
#             if new_jp and new_jp > jp:
#                 print(jp)
#                 jp = new_jp
#             else:
#                 clean_sequence[i] = oc
                
# clean()